In [2]:
import os
import nest_asyncio
import chess
import chess.engine
import asyncio

In [3]:
from database.database.ask_db import *

/home/jon/anaconda3/envs/chessism_gpu/lib/python3.12/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [ ]:
open_request('select * from fen')

In [9]:
get_all_tables()

[('player',), ('game',), ('months',), ('moves',)]

In [8]:
delete_all_leela_tables()

Deleting table: fen...
Successfully deleted table: fen
Database connection closed.
Deleting table: leela_scored...
Successfully deleted table: leela_scored
Database connection closed.


In [2]:
nest_asyncio.apply()
LC0_PATH = "/home/jon/workshop/leela_zero/leela_engine_python/Lc0/lc0.exe"
lc0_directory = os.path.dirname(LC0_PATH)
LC0_WEIGHTS_FILE = "791556.pb.gz" 

In [41]:
async def initialize_lc0_engine() -> chess.engine.UciProtocol:
    engine_uci = None
    try:
        print("...Starting Leela engine...")
        transport, engine_uci = await chess.engine.popen_uci(LC0_PATH, cwd=lc0_directory)
        await engine_uci.configure({
                                    "WeightsFile": LC0_WEIGHTS_FILE,
                                    "Backend": "cuda-fp16", 
                                    "Threads": 1,
                                    "MinibatchSize": 1024 
                                })
        print("...Leela engine ready...")
        return engine_uci
    except Exception as e:
        print(f"Error initializing Lc0 engine: {e}")
        if engine_uci:
            await engine_uci.quit()
        raise

In [5]:
engine = await initialize_lc0_engine()

Launching Lc0 engine...


<UciProtocol (pid=170013)>: stderr >>        _
<UciProtocol (pid=170013)>: stderr >> |   _ | |
<UciProtocol (pid=170013)>: stderr >> |_ |_ |_| v0.31.2 built Oct 20 2024


Lc0 engine successfully launched and configured.


<UciProtocol (pid=170013)>: stderr >> Loading weights file from: 791556.pb.gz
<UciProtocol (pid=170013)>: stderr >> Creating backend [cuda-fp16]...
<UciProtocol (pid=170013)>: stderr >> CUDA Runtime version: 11.1.0
<UciProtocol (pid=170013)>: stderr >> Latest version of CUDA supported by the driver: 12.9.0
<UciProtocol (pid=170013)>: stderr >> GPU: NVIDIA GeForce RTX 4060
<UciProtocol (pid=170013)>: stderr >> GPU memory: 7.99548 Gb
<UciProtocol (pid=170013)>: stderr >> GPU clock frequency: 2490 MHz
<UciProtocol (pid=170013)>: stderr >> GPU compute capability: 8.9
<UciProtocol (pid=170013)>: stderr >> L2 cache capacity: 25165824


In [37]:
async def analyse(engine: chess.engine.UciProtocol,fens: list) -> dict():
    result = {}
    for fen in fens:
        board = chess.Board(fen)
        limit = chess.engine.Limit(nodes=50000)
        info = await engine.analyse(board, limit=limit)
        result[fen] = info
        result[fen]['score'] = info["score"].white().score(mate_score=10000) / 100
        result[fen]['pv'] = [move.uci() for move in result[fen]['pv']]
    return result

In [38]:
positions_to_analyze = [
        "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1", # Starting position
        "r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3", # Ruy Lopez opening
        "8/8/6P1/4R3/8/6k1/2r5/6K1 b - - 0 1", # Complex endgame
        # Add hundreds more FEN strings here as needed
        # "r3k2r/p1qnp1b1/2p2npp/4p3/1pP1P3/3P1N2/PP1NB1PP/R1BQ1RK1 w kq - 0 1", # Example additional FEN
        # "q3r1k1/pp3p1p/4b1p1/8/8/P3PQ2/1P3PPP/R4RK1 b - - 0 1", # Another example
    ]

In [39]:
res = await analyse(engine, positions_to_analyze)

In [40]:
res['rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1']

{'depth': 11,
 'seldepth': 38,
 'time': 0.286,
 'nodes': 41923,
 'score': 0.27,
 'tbhits': 0,
 'pv': ['e2e4',
  'e7e5',
  'g1f3',
  'g8f6',
  'd2d4',
  'f6e4',
  'f1d3',
  'd7d5',
  'f3e5',
  'b8d7',
  'e5d7',
  'c8d7',
  'e1g1',
  'd8h4',
  'c2c4',
  'e8c8',
  'c4c5',
  'g7g6',
  'b1c3',
  'f8g7',
  'g2g3',
  'h4f6',
  'c1e3',
  'e4g5',
  'f2f4',
  'g5h3',
  'g1g2',
  'd8e8',
  'd1d2',
  'h7h5',
  'a1e1'],
 'nps': 155847}

In [3]:
import os
import nest_asyncio
import chess
import chess.engine
import asyncio

# Apply nest_asyncio for Jupyter/IPython environments
nest_asyncio.apply()

# --- Configuration ---
# Set the absolute path to your Lc0 executable within WSL
LC0_PATH = ["/home/jon/workshop/leela_zero/leela_engine_python/Lc0/lc0.exe"]

# Derive the directory from the executable path for Lc0's working directory
lc0_directory = os.path.dirname(LC0_PATH[0])

# Define the path to your Lc0 network weights file
# Assuming it's in the same directory as lc0.exe
LC0_WEIGHTS_FILE = "791556.pb.gz" 
# --- End Configuration ---


async def initialize_lc0_engine() -> chess.engine.UciProtocol:
    """
    Launches and configures the Leela Chess Zero (Lc0) engine once.

    Returns:
        An initialized Lc0 engine instance (chess.engine.UciProtocol).

    Raises:
        Exception: If the engine fails to launch or configure.
    """
    engine_uci = None
    try:
        print("Launching Lc0 engine...")
        # Launch Lc0 engine
        transport, engine_uci = await chess.engine.popen_uci(
            LC0_PATH, 
            cwd=lc0_directory
        )
        
        # Configure Lc0 options
        await engine_uci.configure({
            "WeightsFile": LC0_WEIGHTS_FILE,
            "Backend": "cuda-fp16", # Use cuda-fp16 for optimal performance on RTX 4060
            "Threads": 1,
            "MinibatchSize": 1024   # Max allowed for your Lc0 build
        })
        print("Lc0 engine successfully launched and configured.")
        return engine_uci
    except Exception as e:
        print(f"Error initializing Lc0 engine: {e}")
        # Ensure engine is quit even if configuration fails
        if engine_uci:
            await engine_uci.quit()
        raise # Re-raise the exception to propagate the error


async def analyze_single_position(engine_uci: chess.engine.UciProtocol, fen: str, nodes_limit: int = 50000) -> dict:
    """
    Analyzes a single chess position using an already initialized Leela Chess Zero (Lc0) engine
    and returns its centipawn score and principal variation.

    Args:
        engine_uci: An already initialized Lc0 engine instance.
        fen: The FEN string of the position to analyze.
        nodes_limit: The maximum number of nodes Lc0 should explore (playouts).

    Returns:
        A dictionary containing the FEN, centipawn score, and principal variation (PV).
        Includes an 'error' key if analysis fails.
    """
    board = chess.Board(fen)

    try:
        # Set the search limit (nodes in this case)
        limit = chess.engine.Limit(nodes=nodes_limit)

        # Perform the analysis using the provided engine instance
        info = await engine_uci.analyse(board, limit=limit)

        # Extract the score (centipawns from White's perspective)
        # mate_score is used to give a high/low value for checkmates
        score = info["score"].white().score(mate_score=10000) 
        
        # Extract the principal variation (PV)
        pv = [move.uci() for move in info["pv"]]

        return {"fen": fen, "score": score, "pv": pv}

    except Exception as e:
        print(f"An error occurred during Lc0 analysis for FEN {fen}: {e}")
        return {"fen": fen, "error": str(e)}


async def analyze_fens_batch(engine_uci: chess.engine.UciProtocol, fens: list[str], nodes_limit: int = 50000) -> dict:
    """
    Analyzes a list of FEN positions using a single Lc0 engine instance.

    Args:
        engine_uci: An already initialized Lc0 engine instance.
        fens: A list of FEN strings to analyze.
        nodes_limit: The maximum number of nodes Lc0 should explore for each position.

    Returns:
        A dictionary where keys are FEN strings and values are the centipawn scores (int)
        or an error string if analysis failed for that FEN.
    """
    analysis_results_dict = {}

    print("\nStarting batch analysis...")

    for i, fen in enumerate(fens):
        print(f"[{i+1}/{len(fens)}] Analyzing FEN: {fen}...")
        result = await analyze_single_position(engine_uci, fen, nodes_limit=nodes_limit) 
        
        if "error" in result:
            print(f"Failed to analyze {result['fen']}: {result['error']}")
            analysis_results_dict[result['fen']] = "Error: " + result['error']
        else:
            analysis_results_dict[result['fen']] = result['score']
            # Only print first move of PV for brevity in batch analysis progress
            print(f"  Score: {result['score']} cp, PV: {result['pv'][0]}...")
    
    return analysis_results_dict


async def main():
    """
    Main function to initialize the Lc0 engine, analyze multiple chess positions,
    and print the results.
    """
    positions_to_analyze = [
        "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1", # Starting position
        "r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3", # Ruy Lopez opening
        "8/8/6P1/4R3/8/6k1/2r5/6K1 b - - 0 1", # Complex endgame
        # Add hundreds more FEN strings here as needed
        # "r3k2r/p1qnp1b1/2p2npp/4p3/1pP1P3/3P1N2/PP1NB1PP/R1BQ1RK1 w kq - 0 1", # Example additional FEN
        # "q3r1k1/pp3p1p/4b1p1/8/8/P3PQ2/1P3PPP/R4RK1 b - - 0 1", # Another example
    ]

    # Dictionary to store final results
    final_analysis_results = {}
    engine_uci = None # Initialize engine_uci for outer finally block

    try:
        # Step 1: Initialize the Lc0 engine
        engine_uci = await initialize_lc0_engine()
        
        # Step 2: Analyze the FENs using the initialized engine
        final_analysis_results = await analyze_fens_batch(engine_uci, positions_to_analyze, nodes_limit=50000)

    except Exception as e:
        print(f"An unrecoverable error occurred in the main analysis flow: {e}")
    finally:
        # Ensure the engine is quit after all analyses are done, or if an error occurs
        if engine_uci:
            print("\nQuitting Lc0 engine...")
            await engine_uci.quit()
            print("Lc0 engine quit successfully.")

    print("\n--- Overall Analysis Complete ---")
    print("All Centipawn Scores:")
    for fen, score in final_analysis_results.items():
        print(f"FEN: {fen}")
        print(f"  Score: {score} cp")
    
    print("\nFull Results Dictionary:")
    print(final_analysis_results)


# Run the main asynchronous function
await main()


Launching Lc0 engine...


<UciProtocol (pid=159510)>: stderr >>        _
<UciProtocol (pid=159510)>: stderr >> |   _ | |
<UciProtocol (pid=159510)>: stderr >> |_ |_ |_| v0.31.2 built Oct 20 2024
<UciProtocol (pid=159510)>: stderr >> Loading weights file from: 791556.pb.gz


Lc0 engine successfully launched and configured.

Starting batch analysis...
[1/3] Analyzing FEN: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1...


<UciProtocol (pid=159510)>: stderr >> Creating backend [cuda-fp16]...
<UciProtocol (pid=159510)>: stderr >> CUDA Runtime version: 11.1.0
<UciProtocol (pid=159510)>: stderr >> Latest version of CUDA supported by the driver: 12.9.0
<UciProtocol (pid=159510)>: stderr >> GPU: NVIDIA GeForce RTX 4060
<UciProtocol (pid=159510)>: stderr >> GPU memory: 7.99548 Gb
<UciProtocol (pid=159510)>: stderr >> GPU clock frequency: 2490 MHz
<UciProtocol (pid=159510)>: stderr >> GPU compute capability: 8.9
<UciProtocol (pid=159510)>: stderr >> L2 cache capacity: 25165824


  Score: 26 cp, PV: e2e4...
[2/3] Analyzing FEN: r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3...
  Score: 29 cp, PV: f1b5...
[3/3] Analyzing FEN: 8/8/6P1/4R3/8/6k1/2r5/6K1 b - - 0 1...
  Score: -9998 cp, PV: c2c1...

Quitting Lc0 engine...
Lc0 engine quit successfully.

--- Overall Analysis Complete ---
All Centipawn Scores:
FEN: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
  Score: 26 cp
FEN: r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3
  Score: 29 cp
FEN: 8/8/6P1/4R3/8/6k1/2r5/6K1 b - - 0 1
  Score: -9998 cp

Full Results Dictionary:
{'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1': 26, 'r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3': 29, '8/8/6P1/4R3/8/6k1/2r5/6K1 b - - 0 1': -9998}


In [ ]:
async def analyze_position_with_lc0(engine_uci: chess.engine.UciProtocol, fen: str, nodes_limit: int = 50000) -> dict:
    """
    Analyzes a single chess position using an already initialized Leela Chess Zero (Lc0) engine
    and returns its centipawn score and principal variation.

    Args:
        engine_uci: An already initialized Lc0 engine instance.
        fen: The FEN string of the position to analyze.
        nodes_limit: The maximum number of nodes Lc0 should explore (playouts).

    Returns:
        A dictionary containing the FEN, centipawn score, and principal variation (PV).
        Includes an 'error' key if analysis fails.
    """
    board = chess.Board(fen)

    try:
        # Set the search limit (nodes in this case)
        limit = chess.engine.Limit(nodes=nodes_limit)

        # Perform the analysis using the provided engine instance
        info = await engine_uci.analyse(board, limit=limit)

        # Extract the score (centipawns from White's perspective)
        # mate_score is used to give a high/low value for checkmates
        score = info["score"].white().score(mate_score=10000) 
        
        # Extract the principal variation (PV)
        pv = [move.uci() for move in info["pv"]]

        return {"fen": fen, "score": score, "pv": pv}

    except Exception as e:
        print(f"An error occurred during Lc0 analysis for FEN {fen}: {e}")
        return {"fen": fen, "error": str(e)}
    # Removed the finally block from here, engine will be quit in main()

In [5]:
import os
import nest_asyncio
import chess
import chess.engine
import asyncio

# Apply nest_asyncio for Jupyter/IPython environments
nest_asyncio.apply()

# --- Configuration ---
# Set the absolute path to your Lc0 executable within WSL
LC0_PATH = ["/home/jon/workshop/leela_zero/leela_engine_python/Lc0/lc0.exe"]

# Derive the directory from the executable path for Lc0's working directory
lc0_directory = os.path.dirname(LC0_PATH[0])

# Define the path to your Lc0 network weights file
# Assuming it's in the same directory as lc0.exe
LC0_WEIGHTS_FILE = "791556.pb.gz" 
# --- End Configuration ---


async def analyze_position_with_lc0(engine_uci: chess.engine.UciProtocol, fen: str, nodes_limit: int = 50000) -> dict:
    """
    Analyzes a single chess position using an already initialized Leela Chess Zero (Lc0) engine
    and returns its centipawn score and principal variation.

    Args:
        engine_uci: An already initialized Lc0 engine instance.
        fen: The FEN string of the position to analyze.
        nodes_limit: The maximum number of nodes Lc0 should explore (playouts).

    Returns:
        A dictionary containing the FEN, centipawn score, and principal variation (PV).
        Includes an 'error' key if analysis fails.
    """
    board = chess.Board(fen)

    try:
        # Set the search limit (nodes in this case)
        limit = chess.engine.Limit(nodes=nodes_limit)

        # Perform the analysis using the provided engine instance
        info = await engine_uci.analyse(board, limit=limit)

        # Extract the score (centipawns from White's perspective)
        # mate_score is used to give a high/low value for checkmates
        score = info["score"].white().score(mate_score=10000) 
        
        # Extract the principal variation (PV)
        pv = [move.uci() for move in info["pv"]]

        return {"fen": fen, "score": score, "pv": pv}

    except Exception as e:
        print(f"An error occurred during Lc0 analysis for FEN {fen}: {e}")
        return {"fen": fen, "error": str(e)}
    # Removed the finally block from here, engine will be quit in main()


async def main():
    """
    Main function to analyze multiple chess positions and store results.
    Initializes the Lc0 engine once and reuses it for all analyses.
    """
    positions_to_analyze = [
        "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1", # Starting position
        "r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3", # Ruy Lopez opening
        "8/8/6P1/4R3/8/6k1/2r5/6K1 b - - 0 1", # Complex endgame
        # Add hundreds more FEN strings here as needed
        # "r3k2r/p1qnp1b1/2p2npp/4p3/1pP1P3/3P1N2/PP1NB1PP/R1BQ1RK1 w kq - 0 1", # Example additional FEN
        # "q3r1k1/pp3p1p/4b1p1/8/8/P3PQ2/1P3PPP/R4RK1 b - - 0 1", # Another example
    ]

    # Dictionary to store results: {fen_string: centipawn_score}
    analysis_results_dict = {}

    print("Starting Lc0 analysis for multiple FENs...")

    engine_uci = None # Initialize engine_uci for outer finally block
    try:
        print("Launching Lc0 engine...")
        # Launch Lc0 engine once
        transport, engine_uci = await chess.engine.popen_uci(
            LC0_PATH, 
            cwd=lc0_directory
        )
        
        # Configure Lc0 options once
        await engine_uci.configure({
            "WeightsFile": LC0_WEIGHTS_FILE,
            "Backend": "cuda-fp16", # Use cuda-fp16 for optimal performance on RTX 4060
            "Threads": 1,
            "MinibatchSize": 1024   # Max allowed for your Lc0 build
        })
        print("Lc0 engine successfully launched and configured.")

        # Analyze positions one by one using the same engine instance
        for i, fen in enumerate(positions_to_analyze):
            print(f"[{i+1}/{len(positions_to_analyze)}] Analyzing FEN: {fen}...")
            # Pass the already created engine_uci object
            result = await analyze_position_with_lc0(engine_uci, fen, nodes_limit=50000) 
            
            if "error" in result:
                print(f"Failed to analyze {result['fen']}: {result['error']}")
                analysis_results_dict[result['fen']] = "Error: " + result['error']
            else:
                analysis_results_dict[result['fen']] = result['score']
                print(f"  Score: {result['score']} cp, PV: {result['pv'][0]}...")

    except Exception as e:
        print(f"An error occurred in main analysis loop: {e}")
    finally:
        # Ensure the engine is quit after all analyses are done, or if an error occurs
        if engine_uci:
            print("\nQuitting Lc0 engine...")
            await engine_uci.quit()
            print("Lc0 engine quit successfully.")

    print("\n--- Analysis Complete ---")
    print("All Centipawn Scores:")
    for fen, score in analysis_results_dict.items():
        print(f"FEN: {fen}")
        print(f"  Score: {score} cp")
    
    print("\nFull Results Dictionary:")
    print(analysis_results_dict)


# Run the main asynchronous function
await main()

Starting Lc0 analysis for multiple FENs...
Launching Lc0 engine...


<UciProtocol (pid=148996)>: stderr >>        _
<UciProtocol (pid=148996)>: stderr >> |   _ | |
<UciProtocol (pid=148996)>: stderr >> |_ |_ |_| v0.31.2 built Oct 20 2024
<UciProtocol (pid=148996)>: stderr >> Loading weights file from: 791556.pb.gz


Lc0 engine successfully launched and configured.
[1/3] Analyzing FEN: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1...


<UciProtocol (pid=148996)>: stderr >> Creating backend [cuda-fp16]...
<UciProtocol (pid=148996)>: stderr >> CUDA Runtime version: 11.1.0
<UciProtocol (pid=148996)>: stderr >> Latest version of CUDA supported by the driver: 12.9.0
<UciProtocol (pid=148996)>: stderr >> GPU: NVIDIA GeForce RTX 4060
<UciProtocol (pid=148996)>: stderr >> GPU memory: 7.99548 Gb
<UciProtocol (pid=148996)>: stderr >> GPU clock frequency: 2490 MHz
<UciProtocol (pid=148996)>: stderr >> GPU compute capability: 8.9
<UciProtocol (pid=148996)>: stderr >> L2 cache capacity: 25165824


  Score: 26 cp, PV: e2e4...
[2/3] Analyzing FEN: r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3...
  Score: 30 cp, PV: f1b5...
[3/3] Analyzing FEN: 8/8/6P1/4R3/8/6k1/2r5/6K1 b - - 0 1...
  Score: -9998 cp, PV: c2c1...

Quitting Lc0 engine...
Lc0 engine quit successfully.

--- Analysis Complete ---
All Centipawn Scores:
FEN: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
  Score: 26 cp
FEN: r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3
  Score: 30 cp
FEN: 8/8/6P1/4R3/8/6k1/2r5/6K1 b - - 0 1
  Score: -9998 cp

Full Results Dictionary:
{'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1': 26, 'r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3': 30, '8/8/6P1/4R3/8/6k1/2r5/6K1 b - - 0 1': -9998}


In [3]:
nest_asyncio.apply()
LC0_PATH = ["/home/jon/workshop/leela_zero/leela_engine_python/Lc0/lc0.exe"]
lc0_directory = os.path.dirname(LC0_PATH[0])
LC0_WEIGHTS_FILE = "791556.pb.gz" 

In [4]:
fen = "8/8/6P1/4R3/8/6k1/2r5/6K1 b - - 0 1"

In [ ]:
async def get_engine( nodes_limit: int = 50000):
    

In [2]:
import os
import nest_asyncio
import chess
import chess.engine
import asyncio

# Apply nest_asyncio for Jupyter/IPython environments
nest_asyncio.apply()

# --- Configuration ---
# Set the absolute path to your Lc0 executable within WSL
LC0_PATH = ["/home/jon/workshop/leela_zero/leela_engine_python/Lc0/lc0.exe"]

# Derive the directory from the executable path for Lc0's working directory
lc0_directory = os.path.dirname(LC0_PATH[0])

# Define the path to your Lc0 network weights file
# Assuming it's in the same directory as lc0.exe
LC0_WEIGHTS_FILE = "791556.pb.gz" 
# --- End Configuration ---


async def analyze_position_with_lc0(fen: str, nodes_limit: int = 50000) -> dict:
    """
    Analyzes a single chess position using Leela Chess Zero (Lc0)
    and returns its centipawn score and principal variation.

    Args:
        fen: The FEN string of the position to analyze.
        nodes_limit: The maximum number of nodes Lc0 should explore (playouts).

    Returns:
        A dictionary containing the FEN, centipawn score, and principal variation (PV).
        Includes an 'error' key if analysis fails.
    """
    board = chess.Board(fen)
    engine_uci = None # Initialize to None for the finally block

    try:
        # Launch Lc0 engine, specifying its working directory for weights file
        # The 'engine_uci' variable now holds the actual engine object
        transport, engine_uci = await chess.engine.popen_uci(
            LC0_PATH, 
            cwd=lc0_directory
        )
        
        # Configure Lc0 options for GPU performance
        await engine_uci.configure({
            "WeightsFile": LC0_WEIGHTS_FILE,
            "Backend": "cuda-fp16", # Use cuda-fp16 for optimal performance on RTX 4060
            "Threads": 1,
            "MinibatchSize": 1024   # Max allowed for your Lc0 build
        })

        # Set the search limit (nodes in this case)
        limit = chess.engine.Limit(nodes=nodes_limit)

        # Perform the analysis
        info = await engine_uci.analyse(board, limit=limit)

        # Extract the score (centipawns from White's perspective)
        # mate_score is used to give a high/low value for checkmates
        score = info["score"].white().score(mate_score=10000) 
        
        # Extract the principal variation (PV)
        pv = [move.uci() for move in info["pv"]]

        return {"fen": fen, "score": score, "pv": pv}

    except Exception as e:
        print(f"An error occurred during Lc0 analysis for FEN {fen}: {e}")
        return {"fen": fen, "error": str(e)}
    finally:
        # Always quit the engine process to free up resources
        if engine_uci:
            await engine_uci.quit()


async def main():
    """
    Main function to analyze multiple chess positions and store results.
    """
    positions_to_analyze = [
        "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1", # Starting position
        "r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3", # Ruy Lopez opening
        "8/8/6P1/4R3/8/6k1/2r5/6K1 b - - 0 1", # Complex endgame
        # Add hundreds more FEN strings here as needed
        # "r3k2r/p1qnp1b1/2p2npp/4p3/1pP1P3/3P1N2/PP1NB1PP/R1BQ1RK1 w kq - 0 1", # Example additional FEN
        # "q3r1k1/pp3p1p/4b1p1/8/8/P3PQ2/1P3PPP/R4RK1 b - - 0 1", # Another example
    ]

    # Dictionary to store results: {fen_string: centipawn_score}
    analysis_results_dict = {}

    print("Starting Lc0 analysis for multiple FENs...")

    # Analyze positions one by one
    for i, fen in enumerate(positions_to_analyze):
        print(f"[{i+1}/{len(positions_to_analyze)}] Analyzing FEN: {fen}...")
        result = await analyze_position_with_lc0(fen, nodes_limit=50000) 
        
        if "error" in result:
            print(f"Failed to analyze {result['fen']}: {result['error']}")
            analysis_results_dict[result['fen']] = "Error: " + result['error']
        else:
            analysis_results_dict[result['fen']] = result['score']
            print(f"  Score: {result['score']} cp, PV: {result['pv'][0]}...") # Print only first move of PV for brevity

    print("\n--- Analysis Complete ---")
    print("All Centipawn Scores:")
    for fen, score in analysis_results_dict.items():
        print(f"FEN: {fen}")
        print(f"  Score: {score} cp")
    
    print("\nFull Results Dictionary:")
    print(analysis_results_dict)
    return analysis_results_dict

# Run the main asynchronous function
await main()


Starting Lc0 analysis for multiple FENs...
[1/3] Analyzing FEN: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1...


<UciProtocol (pid=117386)>: stderr >>        _
<UciProtocol (pid=117386)>: stderr >> |   _ | |
<UciProtocol (pid=117386)>: stderr >> |_ |_ |_| v0.31.2 built Oct 20 2024
<UciProtocol (pid=117386)>: stderr >> Loading weights file from: 791556.pb.gz
<UciProtocol (pid=117386)>: stderr >> Creating backend [cuda-fp16]...
<UciProtocol (pid=117386)>: stderr >> CUDA Runtime version: 11.1.0
<UciProtocol (pid=117386)>: stderr >> Latest version of CUDA supported by the driver: 12.9.0
<UciProtocol (pid=117386)>: stderr >> GPU: NVIDIA GeForce RTX 4060
<UciProtocol (pid=117386)>: stderr >> GPU memory: 7.99548 Gb
<UciProtocol (pid=117386)>: stderr >> GPU clock frequency: 2490 MHz
<UciProtocol (pid=117386)>: stderr >> GPU compute capability: 8.9
<UciProtocol (pid=117386)>: stderr >> L2 cache capacity: 25165824


  Score: 26 cp, PV: e2e4...
[2/3] Analyzing FEN: r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3...


<UciProtocol (pid=127784)>: stderr >>        _
<UciProtocol (pid=127784)>: stderr >> |   _ | |
<UciProtocol (pid=127784)>: stderr >> |_ |_ |_| v0.31.2 built Oct 20 2024
<UciProtocol (pid=127784)>: stderr >> Loading weights file from: 791556.pb.gz
<UciProtocol (pid=127784)>: stderr >> Creating backend [cuda-fp16]...
<UciProtocol (pid=127784)>: stderr >> CUDA Runtime version: 11.1.0
<UciProtocol (pid=127784)>: stderr >> Latest version of CUDA supported by the driver: 12.9.0
<UciProtocol (pid=127784)>: stderr >> GPU: NVIDIA GeForce RTX 4060
<UciProtocol (pid=127784)>: stderr >> GPU memory: 7.99548 Gb
<UciProtocol (pid=127784)>: stderr >> GPU clock frequency: 2490 MHz
<UciProtocol (pid=127784)>: stderr >> GPU compute capability: 8.9
<UciProtocol (pid=127784)>: stderr >> L2 cache capacity: 25165824


  Score: 31 cp, PV: f1b5...
[3/3] Analyzing FEN: 8/8/6P1/4R3/8/6k1/2r5/6K1 b - - 0 1...


<UciProtocol (pid=138186)>: stderr >>        _
<UciProtocol (pid=138186)>: stderr >> |   _ | |
<UciProtocol (pid=138186)>: stderr >> |_ |_ |_| v0.31.2 built Oct 20 2024
<UciProtocol (pid=138186)>: stderr >> Loading weights file from: 791556.pb.gz
<UciProtocol (pid=138186)>: stderr >> Creating backend [cuda-fp16]...
<UciProtocol (pid=138186)>: stderr >> CUDA Runtime version: 11.1.0
<UciProtocol (pid=138186)>: stderr >> Latest version of CUDA supported by the driver: 12.9.0
<UciProtocol (pid=138186)>: stderr >> GPU: NVIDIA GeForce RTX 4060
<UciProtocol (pid=138186)>: stderr >> GPU memory: 7.99548 Gb
<UciProtocol (pid=138186)>: stderr >> GPU clock frequency: 2490 MHz
<UciProtocol (pid=138186)>: stderr >> GPU compute capability: 8.9
<UciProtocol (pid=138186)>: stderr >> L2 cache capacity: 25165824


  Score: -9998 cp, PV: c2c1...

--- Analysis Complete ---
All Centipawn Scores:
FEN: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
  Score: 26 cp
FEN: r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3
  Score: 31 cp
FEN: 8/8/6P1/4R3/8/6k1/2r5/6K1 b - - 0 1
  Score: -9998 cp

Full Results Dictionary:
{'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1': 26, 'r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3': 31, '8/8/6P1/4R3/8/6k1/2r5/6K1 b - - 0 1': -9998}


{'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1': 26,
 'r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3': 31,
 '8/8/6P1/4R3/8/6k1/2r5/6K1 b - - 0 1': -9998}

In [1]:
import os
import nest_asyncio
nest_asyncio.apply()
import chess
import chess.engine
import asyncio
import subprocess

# LC0_PATH and lc0_directory definitions (assuming they are now correctly set for WSL)
LC0_PATH = ["/home/jon/workshop/leela_zero/leela_engine_python/Lc0/lc0.exe"]
lc0_directory = os.path.dirname(LC0_PATH[0])

# Define network_files (assuming it's in the same directory as lc0.exe in WSL)
network_files = [os.path.join(lc0_directory, "791556.pb.gz")] # Adjust if your weights file is different

async def analyze_position_with_lc0(fen: str, nodes_limit: int = 10000, time_limit: float = None):
    """
    Analyzes a single chess position using Leela Chess Zero (Lc0).
    """
    board = chess.Board(fen)

    # --- DEBUGGING PRINTS ---
    print(f"DEBUG: LC0_PATH being used: {LC0_PATH}")
    print(f"DEBUG: lc0_directory being used: {lc0_directory}")
    
    executable_path_str = LC0_PATH[0]
    print(f"DEBUG: Does executable file exist at '{executable_path_str}'? {os.path.exists(executable_path_str)}")
    # --- END DEBUGGING PRINTS ---

    # --- Direct subprocess.Popen test (no shell=True) ---
    print("\n--- Starting direct subprocess.Popen test (no shell=True) ---") 
    direct_test_successful = False
    process = None # Initialize process outside try for finally block
    try:
        process = subprocess.Popen(LC0_PATH, cwd=lc0_directory, 
                                   stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                                   text=True)
        
        stdout, stderr = process.communicate(input="uci\nisready\nquit\n", timeout=10) 
        print(f"DEBUG: Direct subprocess stdout:\n{stdout}")
        print(f"DEBUG: Direct subprocess stderr:\n{stderr}")
        
        if "id name Lc0" in stdout: 
            print("DEBUG: Direct subprocess test: Lc0 responded to 'uci' command (Success!).")
            direct_test_successful = True
        else:
            print("DEBUG: Direct subprocess test: Lc0 did not respond to 'uci' command as expected.")

    except FileNotFoundError as fnfe:
        print(f"DEBUG: DIRECT SUBPROCESS POPEN FAILED with FileNotFoundError: {fnfe}")
        return {"fen": fen, "error": f"Direct subprocess test failed: {fnfe}"}
    except subprocess.TimeoutExpired:
        print("DEBUG: Direct subprocess.Popen timed out. Lc0 likely started but didn't respond to 'uci' in time.")
        if process:
            process.kill()
        return {"fen": fen, "error": "Direct subprocess test timed out."}
    except Exception as e_direct_subprocess:
        print(f"DEBUG: Error during direct subprocess.Popen test: {e_direct_subprocess}")
        return {"fen": fen, "error": f"Direct subprocess test encountered an unexpected error: {e_direct_subprocess}"}
    finally:
        if process and process.poll() is None: # Check if process is still running
            process.terminate()
            try:
                process.wait(timeout=1) # Give it a moment to terminate
            except subprocess.TimeoutExpired:
                process.kill()
        print(f"--- Direct subprocess.Popen test ended (return code: {process.returncode if process else 'N/A'}) ---\n")
    
    if not direct_test_successful:
        return {"fen": fen, "error": "Direct Lc0 subprocess test failed, cannot proceed with analysis."}


    # Proceed with chess.engine.popen_uci
    # --- KEY CHANGE HERE: Unpack the tuple returned by popen_uci ---
    transport = None # Initialize transport and engine_uci for finally block
    engine_uci = None
    try:
        transport, engine_uci = await chess.engine.popen_uci(
            LC0_PATH, 
            cwd=lc0_directory
        )
        # 'engine_uci' is the actual engine object we need to work with
        
        # Lc0 specific options
        await engine_uci.configure({ # <--- USE engine_uci here
            "WeightsFile": network_files[0] if network_files else "", 
            "Backend": "cuda-fp16", # This is often the most performant if you have a CUDA GPU
            "Threads": 1,
            "MinibatchSize": 1024
        })

        limit = chess.engine.Limit(nodes=nodes_limit)
        if time_limit:
            limit = chess.engine.Limit(time=time_limit)

        print(f"Analyzing FEN: {fen} with Lc0 (Nodes: {nodes_limit}, Time: {time_limit}s)...")
        info = await engine_uci.analyse(board, limit=limit) # <--- USE engine_uci here

        score = info["score"].white().score(mate_score=10000)
        pv = [move.uci() for move in info["pv"]]

        return {"fen": fen, "score": score, "pv": pv}

    except Exception as e:
        print(f"An error occurred during Lc0 analysis with chess.engine: {e}")
        return {"fen": fen, "error": str(e)}
    finally:
        # --- KEY CHANGE HERE: Quit the engine_uci object ---
        if engine_uci: # Ensure engine_uci was successfully created
            await engine_uci.quit() # Always quit the engine process

# The main() function and the call to await main() remain the same.

import asyncio

async def main():
    positions_to_analyze = [
        "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1", # Starting position
        "r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3", # Ruy Lopez opening
        "8/8/6P1/4R3/8/6k1/2r5/6K1 b - - 0 1", # Complex endgame
    ]

    # Analyze positions one by one
    for fen in positions_to_analyze:
        result = await analyze_position_with_lc0(fen, nodes_limit=50000) # Aim for more nodes with Lc0
        if "error" in result:
            print(f"Failed to analyze {result['fen']}: {result['error']}")
        else:
            print(f"\nFEN: {result['fen']}")
            print(f"  Score (cp from White's perspective): {result['score']}")
            print(f"  Principal Variation (PV): {result['pv']}")
            print("-" * 40)

await main()

DEBUG: LC0_PATH being used: ['/home/jon/workshop/leela_zero/leela_engine_python/Lc0/lc0.exe']
DEBUG: lc0_directory being used: /home/jon/workshop/leela_zero/leela_engine_python/Lc0
DEBUG: Does executable file exist at '/home/jon/workshop/leela_zero/leela_engine_python/Lc0/lc0.exe'? True

--- Starting direct subprocess.Popen test (no shell=True) ---
DEBUG: Direct subprocess stdout:
id name Lc0 v0.31.2
id author The LCZero Authors.
option name WeightsFile type string default <autodiscover>
option name Backend type combo default cuda-auto var cuda-auto var cuda var cuda-fp16 var trivial var random var check var recordreplay var roundrobin var multiplexing var demux
option name BackendOptions type string default 
option name Threads type spin default 0 min 0 max 128
option name NNCacheSize type spin default 2000000 min 0 max 999999999
option name MinibatchSize type spin default 0 min 0 max 1024
option name MaxPrefetch type spin default 32 min 0 max 1024
option name CPuct type string defaul

<UciProtocol (pid=84605)>: stderr >>        _
<UciProtocol (pid=84605)>: stderr >> |   _ | |
<UciProtocol (pid=84605)>: stderr >> |_ |_ |_| v0.31.2 built Oct 20 2024
<UciProtocol (pid=84605)>: stderr >> Loading weights file from: /home/jon/workshop/leela_zero/leela_engine_python/Lc0/791556.pb.gz


Analyzing FEN: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1 with Lc0 (Nodes: 50000, Time: Nones)...


<UciProtocol (pid=84605)>: stderr >> Creating backend [cuda-fp16]...
<UciProtocol (pid=84605)>: stderr >> CUDA Runtime version: 11.1.0
<UciProtocol (pid=84605)>: stderr >> Latest version of CUDA supported by the driver: 12.9.0
<UciProtocol (pid=84605)>: stderr >> GPU: NVIDIA GeForce RTX 4060
<UciProtocol (pid=84605)>: stderr >> GPU memory: 7.99548 Gb
<UciProtocol (pid=84605)>: stderr >> GPU clock frequency: 2490 MHz
<UciProtocol (pid=84605)>: stderr >> GPU compute capability: 8.9
<UciProtocol (pid=84605)>: stderr >> L2 cache capacity: 25165824



FEN: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
  Score (cp from White's perspective): 26
  Principal Variation (PV): ['e2e4', 'e7e5', 'g1f3', 'g8f6', 'd2d4', 'f6e4', 'f1d3', 'd7d5', 'f3e5', 'b8d7', 'e5d7', 'c8d7', 'e1g1', 'd8h4', 'c2c4', 'e8c8', 'c4c5', 'g7g6', 'b1c3', 'f8g7', 'g2g3', 'h4f6', 'c1e3', 'e4g5', 'd3e2', 'd7h3', 'c5c6', 'b7c6', 'a1c1', 'h3f1']
----------------------------------------
DEBUG: LC0_PATH being used: ['/home/jon/workshop/leela_zero/leela_engine_python/Lc0/lc0.exe']
DEBUG: lc0_directory being used: /home/jon/workshop/leela_zero/leela_engine_python/Lc0
DEBUG: Does executable file exist at '/home/jon/workshop/leela_zero/leela_engine_python/Lc0/lc0.exe'? True

--- Starting direct subprocess.Popen test (no shell=True) ---
DEBUG: Direct subprocess stdout:
id name Lc0 v0.31.2
id author The LCZero Authors.
option name WeightsFile type string default <autodiscover>
option name Backend type combo default cuda-auto var cuda-auto var cuda var cuda-fp16 var tr

...


FEN: r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3
  Score (cp from White's perspective): 31
  Principal Variation (PV): ['f1b5', 'a7a6', 'b5a4', 'g8f6', 'e1g1', 'b7b5', 'a4b3', 'f8c5', 'a2a4', 'c8b7', 'd2d3', 'e8g8', 'b1c3', 'c6a5', 'b3a2', 'b5b4', 'c3e2', 'd7d5', 'e4d5', 'b7d5']
----------------------------------------
DEBUG: LC0_PATH being used: ['/home/jon/workshop/leela_zero/leela_engine_python/Lc0/lc0.exe']
DEBUG: lc0_directory being used: /home/jon/workshop/leela_zero/leela_engine_python/Lc0
DEBUG: Does executable file exist at '/home/jon/workshop/leela_zero/leela_engine_python/Lc0/lc0.exe'? True

--- Starting direct subprocess.Popen test (no shell=True) ---
DEBUG: Direct subprocess stdout:
id name Lc0 v0.31.2
id author The LCZero Authors.
option name WeightsFile type string default <autodiscover>
option name Backend type combo default cuda-auto var cuda-auto var cuda var cuda-fp16 var trivial var random var check var recordreplay var roundrobin var multiplex

<UciProtocol (pid=106930)>: stderr >>        _
<UciProtocol (pid=106930)>: stderr >> |   _ | |
<UciProtocol (pid=106930)>: stderr >> |_ |_ |_| v0.31.2 built Oct 20 2024
<UciProtocol (pid=106930)>: stderr >> Loading weights file from: /home/jon/workshop/leela_zero/leela_engine_python/Lc0/791556.pb.gz


Analyzing FEN: 8/8/6P1/4R3/8/6k1/2r5/6K1 b - - 0 1 with Lc0 (Nodes: 50000, Time: Nones)...


<UciProtocol (pid=106930)>: stderr >> Creating backend [cuda-fp16]...
<UciProtocol (pid=106930)>: stderr >> CUDA Runtime version: 11.1.0
<UciProtocol (pid=106930)>: stderr >> Latest version of CUDA supported by the driver: 12.9.0
<UciProtocol (pid=106930)>: stderr >> GPU: NVIDIA GeForce RTX 4060
<UciProtocol (pid=106930)>: stderr >> GPU memory: 7.99548 Gb
<UciProtocol (pid=106930)>: stderr >> GPU clock frequency: 2490 MHz
<UciProtocol (pid=106930)>: stderr >> GPU compute capability: 8.9
<UciProtocol (pid=106930)>: stderr >> L2 cache capacity: 25165824



FEN: 8/8/6P1/4R3/8/6k1/2r5/6K1 b - - 0 1
  Score (cp from White's perspective): -9998
  Principal Variation (PV): ['c2c1', 'e5e1', 'c1e1']
----------------------------------------


In [3]:
kokok

NameError: name 'kokok' is not defined

In [ ]:
uciok
readyok

DEBUG: Direct subprocess stderr:
       _
|   _ | |
|_ |_ |_| v0.31.2 built Oct 20 2024

DEBUG: Direct subprocess test: Lc0 responded to 'uci' command (Success!).
--- Direct subprocess.Popen test ended (return code: 0) ---

<UciProtocol (pid=47980)>: stderr >>        _
<UciProtocol (pid=47980)>: stderr >> |   _ | |
<UciProtocol (pid=47980)>: stderr >> |_ |_ |_| v0.31.2 built Oct 20 2024

An error occurred during Lc0 analysis with chess.engine: expected value for option 'MinibatchSize' to be at most 1024, got: 2048
Failed to analyze r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3: expected value for option 'MinibatchSize' to be at most 1024, got: 2048
DEBUG: LC0_PATH being used: ['/home/jon/workshop/leela_zero/leela_engine_python/Lc0/lc0.exe']
DEBUG: lc0_directory being used: /home/jon/workshop/leela_zero/leela_engine_python/Lc0
DEBUG: Does executable file exist at '/home/jon/workshop/leela_zero/leela_engine_python/Lc0/lc0.exe'? True

In [1]:
import os

#LC0_PATH = r"\\wsl.localhost\Ubuntu-24.04\home\jon\workshop\leela_zero\leela_engine_python\Lc0\lc0.exe"
#LC0_PATH =r"Lc0/lc0.exe"

# Change LC0_PATH to be a list containing the executable path
LC0_PATH = [r"\\wsl.localhost\Ubuntu-24.04\home\jon\workshop\leela_zero\leela_engine_python\Lc0\lc0.exe"]

# Extract the directory from the string inside the list
lc0_directory = os.path.dirname(LC0_PATH[0]) # Use LC0_PATH[0] to access the string

# Extract the directory of lc0.exe for the cwd argument
# lc0_directory = os.path.dirname(LC0_PATH) # This line is critical!

import nest_asyncio
nest_asyncio.apply()
import chess
import chess.engine

async def analyze_position_with_lc0(fen: str, nodes_limit: int = 10000, time_limit: float = None):
    """
    Analyzes a single chess position using Leela Chess Zero (Lc0).

    Args:
        fen: The FEN string of the position to analyze.
        nodes_limit: The maximum number of nodes Lc0 should explore (playouts).
                     Lc0 typically uses 'nodes' instead of 'depth' for its search limit.
        time_limit: The maximum time in seconds Lc0 should spend on analysis.

    Returns:
        A dictionary containing the analysis results (score, principal variation).
    """
    board = chess.Board(fen)
    #engine = await chess.engine.popen_uci(LC0_PATH)
    #engine = await chess.engine.popen_uci(LC0_PATH, cwd=lc0_directory) # <--- THIS IS THE FIX
    engine = await chess.engine.popen_uci(LC0_PATH, cwd=lc0_directory) # <--- UPDATED THIS LINE
    try:
        # Lc0 specific options (these are common and good starting points)
        # Check Lc0's documentation for all available options
        await engine.configure({
            "Backend": "cuda",  # Use CUDA with half-precision for speed (requires cuDNN)
                                     # Other options: "cuda", "cpu", "opencl"
            "WeightsFile": network_files[0] if network_files else "", # Lc0 usually picks newest, but explicit is good
            "Threads": 1,            # Lc0 itself is heavily GPU-parallel; 1 thread is often fine for MCTS.
                                     # More threads might be useful if you're doing heavy CPU preprocessing
                                     # or if your Lc0 build benefits from it for specific backends.
            "MinibatchSize": 2048    # Adjust based on your GPU's VRAM. Larger can be faster.
        })

        limit = chess.engine.Limit(nodes=nodes_limit)
        if time_limit:
            limit = chess.engine.Limit(time=time_limit)

        print(f"Analyzing FEN: {fen} with Lc0 (Nodes: {nodes_limit}, Time: {time_limit}s)...")
        info = await engine.analyse(board, limit=limit)

        # Lc0 returns score as "cp" (centipawns) or "mate"
        score = info["score"].white().score(mate_score=10000) # Use a large value for mate scores
        pv = [move.uci() for move in info["pv"]]

        return {"fen": fen, "score": score, "pv": pv}

    except Exception as e:
        print(f"An error occurred during Lc0 analysis: {e}")
        return {"fen": fen, "error": str(e)}
    finally:
        await engine.quit() # Always quit the engine process

import asyncio

async def main():
    positions_to_analyze = [
        "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1", # Starting position
        "r1bqkbnr/pppp1ppp/2n5/4p3/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3", # Ruy Lopez opening
        "8/8/6P1/4R3/8/6k1/2r5/6K1 b - - 0 1", # Complex endgame
    ]

    # Analyze positions one by one
    for fen in positions_to_analyze:
        result = await analyze_position_with_lc0(fen, nodes_limit=50000) # Aim for more nodes with Lc0
        if "error" in result:
            print(f"Failed to analyze {result['fen']}: {result['error']}")
        else:
            print(f"\nFEN: {result['fen']}")
            print(f"  Score (cp from White's perspective): {result['score']}")
            print(f"  Principal Variation (PV): {result['pv']}")
            print("-" * 40)

await main()

FileNotFoundError: [Errno 2] No such file or directory: ''

In [ ]:

if __name__ == "__main__":
    # Ensure your NVIDIA drivers and CUDA are correctly installed
    # PyTorch can be used to quickly verify CUDA setup:
    try:
        import torch
        if torch.cuda.is_available():
            print(f"PyTorch reports CUDA is available. GPU: {torch.cuda.get_device_name(0)}")
        else:
            print("PyTorch reports CUDA is NOT available. Lc0 might run on CPU, but slower.")
    except ImportError:
        print("PyTorch not installed. Cannot verify CUDA availability via PyTorch.")

    asyncio.run(main())